In [1]:
!pip install pandas
!pip install scikit-learn 
!pip install matplotlib

You should consider upgrading via the 'C:\Users\DJ-ACER\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\DJ-ACER\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\DJ-ACER\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
df = pd.read_csv('Monthly_Gold_Price_on_World.csv', usecols=['Date', 'Indonesian rupiah (IDR)'])
df

,Date,Indonesian rupiah (IDR)
0,1979-01-31,139739.28
1,1979-02-28,154683.13
2,1979-03-30,148844.15
3,1979-04-30,152453.63
4,1979-05-31,171656.12
...,...,...
513,2021-10-29,25064431.24
514,2021-11-30,25843517.60
515,2021-12-31,25737878.52
516,2022-01-31,25819285.50


In [3]:
from datetime import datetime

def moddate(tanggal_awal):
    if isinstance(tanggal_awal, pd.Timestamp):
        # Convert Timestamp object to string
        tanggal_str = tanggal_awal.strftime('%Y-%m-%d')


        # Ubah string tanggal menjadi objek datetime
        tanggal_obj = datetime.strptime(tanggal_str, '%Y-%m-%d')
    else:
        # Ubah string tanggal menjadi objek datetime
        tanggal_obj = datetime.strptime(tanggal_awal, '%Y-%m-%d')


    # Memformat tanggal menjadi 'YYYY-MM'
    return tanggal_obj.strftime('%Y-%m')


df['Date'] = df['Date'].apply(moddate)
df['Date']

0      1979-01
1      1979-02
2      1979-03
3      1979-04
4      1979-05
        ...   
513    2021-10
514    2021-11
515    2021-12
516    2022-01
517    2022-02
Name: Date, Length: 518, dtype: object